In [29]:
using LowLevelFEM
gmsh.initialize()

In [30]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000721815s, CPU 0.000721s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0158417s, CPU 0.015763s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [31]:
mat = Material("body")
prob = Problem([mat], type=:PlaneStress)

Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [32]:
K = stiffnessMatrix(prob)
M = massMatrix(prob)

sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  187, 188, 189, 190, 191, 192, 193, 194, 195, 196], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  187, 188, 189, 190, 191, 192, 193, 194, 195, 196], [2.993020228862359e-11, 2.993020228862359e-11, 2.993020228862358e-11, 2.993020228862358e-11, 2.993020228862359e-11, 2.993020228862359e-11, 2.99302022886236e-11, 2.99302022886236e-11, 4.420508840131675e-11, 4.420508840131675e-11  …  6.513183780583524e-11, 6.513183780583524e-11, 6.57312862682614e-11, 6.57312862682614e-11, 6.810895857026154e-11, 6.810895857026154e-11, 5.4986235500860704e-11, 5.4986235500860704e-11, 7.466906578570302e-11, 7.466906578570302e-11], 196, 196)

In [33]:
supp = BoundaryCondition("left", ux=0, uy=0)

BoundaryCondition("left", 0, 0, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [34]:
Tmin = smallestPeriodTime(K, M, [supp])
Δt = Tmin / π

1.5074786152874593e-8

In [35]:
function fu(x, y, z, t)
    if t < 2Δt
        return 0.0
    elseif t < 4Δt
        return 1.0e-6
    else
        return 0.0
    end
end
function ff(x, y, z, t)
    if t < 50Δt
        return 0.0
    elseif t < 52Δt
        return 1.0
    else
        return 0.0
    end
end
su = ScalarField(prob, "right", fu, steps=100, tmax=100Δt)
sf = ScalarField(prob, "right", ff, steps=100, tmax=100Δt)

supp2 = BoundaryCondition("right", ux=su)
load = BoundaryCondition("right", fy=sf)

f = loadVector(prob, [load])
u0 = vectorField(prob, "body", [0, 0, 0])
v0 = copy(u0)


VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [36]:
showDoFResults(u0, name="u0")
showDoFResults(f, name="f")

1

In [37]:
u, v = CDM(K, M, f, u0, v0, 100, Tmin / π, support=[supp, supp2])

(VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 4.518617954450206e-7 5.287993321567948e-7; 0.0 0.0 … 9.80450310071431e-7 5.028978298722662e-7], [0.0, 1.5074786152874593e-8, 3.0149572305749186e-8, 4.5224358458623776e-8, 6.029914461149837e-8, 7.537393076437297e-8, 9.044871691724757e-8, 1.0552350307012216e-7, 1.2059828922299674e-7, 1.3567307537587133e-7  …  1.3567307537587117e-6, 1.3718055399115862e-6, 1.3868803260644607e-6, 1.4019551122173352e-6, 1.4170298983702097e-6, 1.4321046845230842e-6, 1.4471794706759587e-6, 1.4622542568288332e-6, 1.4773290429817077e-6, 1.4924038291345822e-6], Int64[], 100, :v2D, Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))), VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 

In [38]:
showDoFResults(u, :uvec, visible=true)

2

In [39]:
showDoFResults(v, :vvec)

3

In [40]:
e1 = solveStrain(u)

TensorField
[[0.0 0.0 … -3.597969946115708e-6 -1.2145520273976476e-6; 0.0 0.0 … -9.659529787857303e-7 -3.975096403202954e-6; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 2.0177406053193843e-6 5.893283907035754e-7], [0.0 0.0 … -1.413375399679047e-6 -2.3347679442811284e-6; 0.0 0.0 … -2.8149201512203453e-6 -2.51124946971081e-6; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 8.396072179553425e-7 1.8667560769953204e-6], [0.0 0.0 … -3.4543986116874338e-6 -6.034887487037693e-6; 0.0 0.0 … -1.7676329615033939e-6 -1.1291701491785744e-6; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 1.5815831032576027e-6 2.4042382916588564e-6], [0.0 0.0 … -8.849433078258506e-7 7.943129598649175e-7; 0.0 0.0 … 1.3629229623822407e-8 5.300760544813468e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -2.907516671307906e-7 1.1829682908281987e-7], [0.0 0.0 … -3.0182926036564708e-6 -4.8597810042213345e-6; 0.0 0.0 … -5.055923050079412e-7 -6.655393025631785e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 1.0708542793671339e-7 2.648064521324181e-6], [0.0 0.0 … 1.040213869953

In [41]:
showStrainResults(e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


4

In [42]:
e2 = solveStrain(u, DoFResults=true)

TensorField
[0.0 0.0 … -2.8318980450185987e-6 -1.6266504181608314e-6; 0.0 0.0 … -1.8588852603533317e-6 -1.8730830250277825e-6; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -7.650713063921957e-8 -3.355957006423809e-7]

In [43]:
showDoFResults(e2, :e)

5

In [44]:
e3 = elementsToNodes(e1)

TensorField
[0.0 0.0 … -2.8318980450185987e-6 -1.6266504181608314e-6; 0.0 0.0 … -9.294426301766659e-7 -9.365415125138913e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -7.650713063921957e-8 -3.355957006423809e-7]

In [45]:
showDoFResults(e3, :e)

6

In [46]:
s1 = solveStress(u)

TensorField
[[0.0 0.0 … -0.8639554694515527 -0.27752006432326504; 0.0 0.0 … -0.14860815058242002 -0.6115532928004545; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.3466127104052907 -0.6463883109656074; 0.0 0.0 … -0.4330646386492839 -0.38634607226320156; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.7747664176838517 -1.298327042876392; 0.0 0.0 … -0.2719435325389837 -0.17371848448901145; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.09141409856847077 0.10400248165878424; 0.0 0.0 … 0.0020968045575111395 0.0815501622278995; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.4808273894758745 -1.155053157776233; 0.0 0.0 … -0.07778343153968326 -0.1023906619327967; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.21172525733593114 0.10058649981940884; 0.0 0.0 … -0.16616073118390867 0.04958637602008401; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.13488377258104894 -0.25432565205092794; 0.0 0.0 … -0.5488377325041949 -0.5918343084012366; 

In [47]:
showStressResults(s1, :s)

7

In [48]:
s2 = solveStress(u, DoFResults=true)

TensorField
[0.0 0.0 … -0.5961230687957342 -0.36978185251484846; 0.0 0.0 … -0.1429911738733332 -0.14408330961752172; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [49]:
showDoFResults(s2, :s)

8

In [50]:
err1 = fieldError(e1)

TensorField
[0.0 0.0 … 5.346033730889346e-7 6.588493976519681e-7; 0.0 0.0 … 6.807154988111888e-8 2.3632968085643475e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.715200965588098e-7 1.2954469119124465e-6]

In [51]:
showDoFResults(err1, :tensor)

9

In [52]:
u2, v2 = HHT(K, M, f, [supp, supp2], u0, v0, 100, Tmin / π)

(VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 2.73980139498332e-7 3.170799366195715e-7; 0.0 0.0 … 9.094107764146525e-7 9.507007797247906e-7], [0.0, 1.5074786152874593e-8, 3.0149572305749186e-8, 4.5224358458623776e-8, 6.029914461149837e-8, 7.537393076437297e-8, 9.044871691724757e-8, 1.0552350307012216e-7, 1.2059828922299674e-7, 1.3567307537587133e-7  …  1.3567307537587117e-6, 1.3718055399115862e-6, 1.3868803260644607e-6, 1.4019551122173352e-6, 1.4170298983702097e-6, 1.4321046845230842e-6, 1.4471794706759587e-6, 1.4622542568288332e-6, 1.4773290429817077e-6, 1.4924038291345822e-6], Int64[], 100, :v2D, Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))), VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 

In [53]:
showDoFResults(u2, :uvec)

10

In [54]:
showDoFResults(v2, :vvec)

11

In [55]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [56]:
gmsh.finalize()